In [7]:
from pymilvus import MilvusClient, Collection, utility, connections

from pymilvus import connections, db, FieldSchema, CollectionSchema, DataType

dbName = "Profspective"
collectionName = "ProfReview"

In [8]:
connection = connections.connect(host="127.0.0.1", port=19530)


# #Delete The Database
# existingDb = db.list_database()
# if(dbName in existingDb):
#     db.drop_database(dbName)
 
existingDb = db.list_database()

if(dbName in existingDb):
    db.using_database(dbName)
else:
    db.create_database(dbName)

In [9]:
client = MilvusClient(uri="http://localhost:19530")

In [10]:
DIMENSIONS = 768
MAX_STRING_LENGTH = 500

fields = [
	FieldSchema(name = "id", dtype= DataType.INT64, is_primary = True, description = "Id"),
 	FieldSchema(name = "embedding", dtype=DataType.FLOAT_VECTOR, dim = DIMENSIONS, description = "Embeddings"),
  	FieldSchema(name="professor", dtype=DataType.VARCHAR, max_length=MAX_STRING_LENGTH, description = "Prof Name"),
    FieldSchema(name="review", dtype=DataType.VARCHAR, max_length=MAX_STRING_LENGTH, default_value = "" , description = "Prof Reivew"),
    FieldSchema(name="stars", dtype=DataType.INT8, default_value = 5, description = "Prof Stars"),
]

schema = CollectionSchema(fields = fields, enable_dynamic_field = True, auto_id = True, description="Schema For Professor Review")

In [19]:
index_params = client.prepare_index_params()

index_params.add_index(
    field_name = "embedding",
	index_type = "IVF_FLAT",
 	metric_type = "L2",
 	params = {"nlist": 40},
)

In [12]:
# '''Used to Clear the Collection (deletes collection and all data)'''

# if client.has_collection("ProfReview"):
#     print("done")
#     client.drop_collection(collection_name = "ProfReview")
 	

In [13]:
def dropIndex():
    fieldName = "embedding"
    existingFieldNames = client.list_indexes(collection_name = collectionName)
    client.release_collection(collection_name = collectionName)
    if(fieldName in existingFieldNames):
        client.drop_index(collection_name = collectionName, index_name = fieldName)

In [20]:
# # Drop the Index
# dropIndex()

client.create_collection(collection_name = collectionName, schema = schema, index_params = index_params)

In [15]:
from sentence_transformers import SentenceTransformer

embedder = SentenceTransformer("nomic-ai/nomic-embed-text-v1.5", trust_remote_code=True)
# embedder = SentenceTransformer("all-MiniLM-L6-v2")

ImportError: This modeling file requires the following packages that were not found in your environment: einops. Run `pip install einops`

In [ ]:
def reviewExists(professor, review, subject, stars):
    queryFilter = f'professor == "{professor}" && review == "{review}" && subject == "{subject}" && stars == {stars}'
    result = client.query(collection_name="ProfReview", filter=queryFilter, output_fields=["id"])
    
    return len(results) > 0

In [ ]:
def validateProfData(profData):
    requiredField = ["professor", "review", "subject", "stars"]
    
    for field in requiredField:
        if field not in profData:
            raise Exception(f'Missing Field in Prof Data "{field}"')

In [ ]:
import json

jsonData = json.load(open("../review.json"))
reviews = jsonData["reviews"]

In [ ]:
insertDb = []

for profData in reviews:
    validateProfData(profData)
    
    if(reviewExists(profData["professor"], profData["review"], profData["subject"], profData["stars"])):
        print("Review Already Exists in Vector Database")
        continue
    
    entry = profData.copy()
    
    reviewEmbedding = embedder.encode(profData["review"])
    print(len(reviewEmbedding))
    
    entry["embedding"] = reviewEmbedding
    
    insertDb.append(entry)
    
if(insertDb):
    output = collection.insert(insertDb)
    print(output)
else:
    print("Nothing to Add")

Review Already Exists in Vector Database
Review Already Exists in Vector Database
Review Already Exists in Vector Database
Review Already Exists in Vector Database
Review Already Exists in Vector Database
Review Already Exists in Vector Database
Review Already Exists in Vector Database
Review Already Exists in Vector Database
Review Already Exists in Vector Database
Review Already Exists in Vector Database
Review Already Exists in Vector Database
Review Already Exists in Vector Database
Review Already Exists in Vector Database
Review Already Exists in Vector Database
Review Already Exists in Vector Database
Review Already Exists in Vector Database
Review Already Exists in Vector Database
Review Already Exists in Vector Database
Review Already Exists in Vector Database
Review Already Exists in Vector Database
Review Already Exists in Vector Database
Review Already Exists in Vector Database
Review Already Exists in Vector Database
Review Already Exists in Vector Database
Review Already E

In [ ]:
collection.flush()

In [ ]:
vector_count = collection.num_entities
print(f'Vector Count = {vector_count}')

Vector Count = 25
